In [ ]:
from index import BS4Parser, Document, Image
test = BS4Parser()

In [4]:
model = test.process_local_file("data/BULLETINS/67068.htm")
model.images = [Image(url="lol", legende="aha"), Image(url="OmO", legende="UwU")]
model

Document(numero='258', date=None, rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact=None, texte=None, images=[Image(url='lol', legende='aha'), Image(url='OmO', legende='UwU')])

In [5]:
xml = model.model_dump_xml_str()
xml

b'<Document><numero>258</numero><rubrique>Focus</rubrique><titre>Physique : Mathias Fink, un bel exemple de chercheur qui innove</titre><auteur>ADIT - Jean-Fran\xc3\xa7ois Desessard - email : jfd@adit.fr</auteur><images><Image><url>lol</url><legende>aha</legende></Image><Image><url>OmO</url><legende>UwU</legende></Image></images></Document>'

In [6]:
import xml.etree.ElementTree as ET
ET.fromstring(xml)

<Element 'Document' at 0x0000016F32CBA390>

In [7]:
Document.model_validate_xml(xml)

Document(numero='258', date=None, rubrique='Focus', titre='Physique : Mathias Fink, un bel exemple de chercheur qui innove', auteur='ADIT - Jean-François Desessard - email : jfd@adit.fr', contact=None, texte=None, images=[Image(url='lol', legende='aha'), Image(url='OmO', legende='UwU')])